In [3]:
import numpy as np
#from counting_grid import CountingGrid

In [152]:
class CountingGrid():
    def __init__(self, size, window_size, nr_of_features):
        '''
        size-- a two-dimensional numpy array, indicating the size of the
        Counting Grid
        window_size-- a two-dimensional numpy array, indicating the size
        of the window
        '''
        np.random.seed(7)
        self.size = size
        self.window_size = window_size
        self.nr_of_features = nr_of_features # No. of features
        #Initialize arrays here for pi, h
        #pi is a 3D array, storing the probability distributions
        #that collectively characterize the data. The first dimensionality
        #corresponds to features, e.g. it is Z. The second and third
        #dimension correspond to the size of the grid in x and y directions.
        rand_init_pi = 1 + np.random.rand(self.nr_of_features,self.size[0],self.size[1])        
        self.pi = rand_init_pi/sum(rand_init_pi,0)
        #Test pi    
        #self.pi = np.array([[[0.494, 0.524],[0.479,0.418]],[[0.506, 0.476],[0.521, 0.582]]])       
        self.h = np.zeros((nr_of_features,self.size[0],self.size[1]))    
        #self.compute_histograms()
        
    def compute_sum_in_a_window(self,grid,k,l):
        cumsum1=np.sum(np.sum(grid[:,:k+self.window_size[0],:l+self.window_size[1]],axis=1),axis=1)
        
        cumsum2=np.sum(np.sum(grid[:,:k,:l+self.window_size[1]],axis=1),axis=1)
        
        cumsum3=np.sum(np.sum(grid[:,:k+self.window_size[0],:l],axis=1),axis=1)
        cumsum4=np.sum(np.sum(grid[:,:k,:l],axis=1),axis=1)
        cumsum=cumsum1-cumsum2-cumsum3+cumsum4
        '''
        cumsum1 = grid[self.window_size[0]-1:,self.window_size[1]-1:]
        cumsum2  = grid[:grid.shape[0]-self.window_size[0]+1,self.window_size[1]-1:]
        cumsum3 = grid[self.window_size[0]-1:,:grid.shape[1]-self.window_size[1]+1]    
        cumsum4 = grid[:grid.shape[0]-self.window_size[0]+1,:grid.shape[1]-self.window_size[1]+1]    
        #print cumsum1.shape, cumsum2.shape,cumsum3.shape    
        cumsum =  cumsum1+cumsum2+cumsum3+cumsum4
        cumsum = cumsum[:cumsum.shape[1]-1,:cumsum.shape[1]-1]
        '''
        return cumsum
    
    def compute_histograms(self):
        '''
        Histograms at each point in the grid are computed
        by averaging the distributions pi in a pre-defined
        window. The left upmost corner of the window is placed
        on the grid position and the distributions are averaged.
        '''
        
    
class CountingGridTest():
    def test_h_computation(self):
        self.pi
        return 0
    def test_e_step(self):
        return 0
    
    def test_m_step(self):
        return 0
    
    def test_arr_sum(self):
        return 0

In [155]:
cg_obj=CountingGrid(np.array([5,5]),np.array([2,2]),3)
#X has dimensions different from the counts in 
#MATLAB implementation-- one row is one data vector,
#that's how it is in sci-kit learn
X=np.array([[1,2,3],[3,4,5]])
print(cg_obj.compute_sum_in_a_window(cg_obj.pi,1,1))
print(cg_obj.pi)
#np.sum(cg_obj.pi[0,:2,:2])
print(np.sum(cg_obj.pi[2,1:3,1:3]))

[1.33729928 1.3825278  1.28017292]
[[[0.29635248 0.37632129 0.29822722 0.35062672 0.40694111]
  [0.41083835 0.36754795 0.25808737 0.29704539 0.3287822 ]
  [0.30684227 0.34322282 0.36844113 0.27003406 0.30988956]
  [0.41534963 0.26777659 0.28728922 0.37499452 0.25905392]
  [0.31010278 0.4339183  0.27675492 0.36939715 0.35406063]]

 [[0.31200873 0.32208919 0.36291471 0.33954886 0.30196772]
  [0.32174166 0.36501264 0.33039017 0.34598054 0.29941042]
  [0.335839   0.33654546 0.35057953 0.39839378 0.30697948]
  [0.31600325 0.29858275 0.32790107 0.26608976 0.3688009 ]
  [0.33311521 0.31440118 0.42884726 0.28160045 0.32290215]]

 [[0.39163879 0.30158952 0.33885807 0.30982442 0.29109117]
  [0.26742    0.26743942 0.41152245 0.35697407 0.37180737]
  [0.35731873 0.32023171 0.28097934 0.33157215 0.38313096]
  [0.26864712 0.43364066 0.3848097  0.35891572 0.37214518]
  [0.35678201 0.25168052 0.29439782 0.3490024  0.32303722]]]
1.2801729225906024
